In [17]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import numpy.matlib as ml


np.set_printoptions(precision=4)
np.set_printoptions(linewidth=150)

In [18]:
A = np.array([[1,0,0,0,0,0],
     [0,2,0,0,0,0],
     [0,0,3,0,0,0],
     [0,0,0,4,0,0],
     [0,0,0,0,5,0],
     [0,0,0,0,0,6],
    ])
x = np.array([[0.3804],
              [0.5678],
              [0.0759],
              [0.0540],
              [0.5308],
              [0.7792]
    ])

m = 8
a = 2.5
b = 5.5
a0 = 1
expected =  [ [0.3804], [0.07060463], [-0.00076315], [0.0005567], [-0.00533705], [0.09689174]]

In [19]:
def chebyshev_filter(A, x,m,a,b,a0):
    e = (b - a)/2
    c = (b + a)/2
    sigma = e/(a0 - c)
    sigma_1 = sigma;
    y = (np.dot(A,x) - c*x)*sigma_1/e
    for i in range (1,m):
        sig_new = 1 / (2/sigma_1 - sigma)
        y_new = 2*(np.dot(A,y) - c*y)*sig_new/e - sigma*sig_new*x
        x = y
        y = y_new
        sigma = sig_new;
    return y

chebyshev_filter_out = chebyshev_filter(A, x,m,a,b,a0)
print(all(abs(chebyshev_filter_out - expected) < 1e-5))

False


In [20]:
B = np.array([[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0]])
display(x, x.shape)
display(x.T, x.T.shape)
display(B)
display(B.T)
display(B[0,:])
display(np.dot(B,x))
display(np.dot(x.T,B))

array([[0.3804],
       [0.5678],
       [0.0759],
       [0.054 ],
       [0.5308],
       [0.7792]])

(6, 1)

array([[0.3804, 0.5678, 0.0759, 0.054 , 0.5308, 0.7792]])

(1, 6)

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

array([[1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

array([1, 0, 0, 0, 0, 0])

array([[0.3804],
       [0.3804],
       [0.3804],
       [0.3804],
       [0.3804],
       [0.3804]])

array([[2.3881, 0.    , 0.    , 0.    , 0.    , 0.    ]])

In [33]:
def cheby_dav_eigh(A, x_vec, m, kwant, kkeep, tol):
    eig_val = np.zeros(A.shape[1])
    kc = 0
    iter_max = 100000
    dim_max = max(kwant * 2,20)
    V = np.zeros((A.shape[0], dim_max),dtype=float)
    W = np.zeros((A.shape[0], dim_max), dtype=float)
    H = np.zeros((dim_max, dim_max),dtype=float)
    
    x = x_vec/la.norm(x_vec)
    V[:,0] = x[:,0]
    w = np.dot(A,x)
    W[:,0] = w[:,0]
    
    mu = np.dot(x.T,w)[0,0]
    H[0,0] = mu
    r = W[:,0].reshape(W.shape[0],1) - mu*x
    
    if la.norm(r) <= tol:
        eig_val[0] = mu
        kc = 1
        H = np.array([])
    else:
        kc= 0
    #print('H',H)
    
    upperb = np.max([np.sum(A[:,i]) for i in range(A.shape[1])])
    lowerb = (upperb+mu)/2
    a0 = 0 #lowerb
    
    print('eig_val', eig_val)
    
    ksub = 0
    iter = 0
    while iter <= iter_max:
        #print('#############################################')
        iter+=1
        if iter % 1000 == 0:
            print('ITERATION',iter)
        #print('kc',kc, 'upperb',upperb,'lowerb',lowerb,'a0',a0)
        t = chebyshev_filter(A, x, m, lowerb, upperb, a0)
        #print('t', t)
        #print('norm(t)', la.norm(t))
        #print('V',V)
        #orthogonalize t against first k_sub columns of V by householder
        for c in range(ksub+1):
            v  = V[:,c]
            v = v.reshape(v.size,1)
            t = t - v * np.dot(v.T, t)[0]/np.dot(v.T,v)
        #print('ortho t', t)
        t = t/la.norm(t)
        #print('normalized t', t)
        #print('ksub', ksub)
        #print('V[:,ksub+1] = t[:,0]', V[:,ksub+1], t[:,0])
        V[:,ksub+1] = t[:,0]
        #print('V after',V)
        kold = ksub
        ksub = ksub + 1;
        W[:,ksub] = A @ V [:, ksub]
        H[kc : ksub +1, ksub] = V[:, kc : ksub+1].T @ W[:, ksub]
        H[ksub, kc : ksub +1] = H[kc : ksub +1, ksub].T
        #print('W\n',W)
        print('H\n',H[0 : ksub +1,0 : ksub +1])
        eigVal,Y = la.eigh(H[kc : ksub +1,kc : ksub +1])
        print("EIGEN_VALUE_CALCULATION")
        print(H[kc : ksub +1,kc : ksub +1])
        print(eigVal)
        print(Y)
        D = np.diag(eigVal)
        print('ksub', ksub,'kc',kc)
        print('D',D)
        print('Eig_vec\n',Y)
        print('V',V[:7,:7])
        #print('Eig_val\n',eigVal)
        if ksub >= dim_max:
            ksub = kc + kkeep
            print('updating ksub to',ksub)
    
        mu = eigVal[0]
        V[:, kc : ksub + 1] = V[:, kc : ksub + 1] @ Y[:, 0 : ksub - kc + 1]
        
        W[:, kc : ksub + 1] = W[:, kc : ksub + 1] @ Y[:, 0 : ksub - kc + 1]
        print('H=D:\n',H[0 : ksub+1, 0 : ksub+1],'\n', D[0 : ksub+1, 0 : ksub+1])
        H[kc : ksub+1, kc : ksub+1] = D[0 : ksub+1, 0 : ksub+1]
        print("H before Convergence\n",H[:7,:7])
        print('V\n',V[:7,:7],'\nW\n',W[:7,:7])
        print(W[:,kc])
        print(mu*V[:,kc])
        r = W[:, kc] - mu * V[:, kc]
        print('norm(r)',la.norm(r),'\n')
        
        noswap = 0
        
        for converge_i in range(ksub-kc+1):
            if la.norm(r) <= tol * max(np.diag(D)):
                print("@@@@@@@@@@@@@@@@@@@@@     CONVERGED:",kc, mu)

                eig_val[kc] = mu
                vtmp = V[:,kc]
                print('eig_val', eig_val)

                i = kc
                while eig_val[i-1] > mu:
                    eig_val[i] = eig_val[i-1]
                    V[:,i] = V[:,i-1]
                    noswap=1 
                    i -= 1
                eig_val[i] = mu
                if noswap == 1:
                    V[:,i] = vtmp
                print("KC, KWANT, NOSWAP")
                print(kc, kwant, noswap)
                kc = kc + 1
                if (kc >= kwant and noswap == 0):
                    return eig_val[0 : kc], V[:, 0:kc]
                mu = eigVal[converge_i+1]
                r = W[:, kc] - mu * V[:, kc]
            else:
                break
        #print('diag(D)\n',np.diag(D))
        lowerb = np.median(np.diag(D[:kc+1,:kc+1]))
        #print('updated loweb', lowerb)
        if a0 > min(np.diag(D)):
            a0 = min(np.diag(D))
            
        x = V[:, kc].reshape(v.shape[0],1)
        #H = np.zeros((dim_max, dim_max),dtype=float)
        #print(H[kc : ksub+1, kc : ksub+1])
        #print(D[kc : ksub+1, kc : ksub+1])
        
    return eig_val
        
x_vec = x

tol = 1e-8
cheby_dav_eigh(A, x_vec, m, 4, 2, tol)

eig_val [0. 0. 0. 0. 0. 0.]
H
 [[ 4.3015 -1.4062]
 [-1.4062  1.6443]]
EIGEN_VALUE_CALCULATION
[[ 4.3015 -1.4062]
 [-1.4062  1.6443]]
[1.0384 4.9074]
[[-0.3957 -0.9184]
 [-0.9184  0.3957]]
ksub 1 kc 0
D [[1.0384 0.    ]
 [0.     4.9074]]
Eig_vec
 [[-0.3957 -0.9184]
 [-0.9184  0.3957]]
V [[ 0.3256  0.9283  0.      0.      0.      0.      0.    ]
 [ 0.486  -0.0017  0.      0.      0.      0.      0.    ]
 [ 0.065  -0.0277  0.      0.      0.      0.      0.    ]
 [ 0.0462 -0.0212  0.      0.      0.      0.      0.    ]
 [ 0.4544 -0.2084  0.      0.      0.      0.      0.    ]
 [ 0.667  -0.3059  0.      0.      0.      0.      0.    ]]
H=D:
 [[ 4.3015 -1.4062]
 [-1.4062  1.6443]] 
 [[1.0384 0.    ]
 [0.     4.9074]]
H before Convergence
 [[1.0384 0.     0.     0.     0.     0.     0.    ]
 [0.     4.9074 0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.

(array([1., 2., 3., 4.]),
 array([[-1.0000e+00, -6.2321e-13, -4.9265e-11,  5.9928e-12],
        [-3.1159e-13,  1.0000e+00,  3.6512e-16,  9.4983e-16],
        [ 1.6422e-11,  3.3254e-16, -1.0000e+00, -5.2218e-17],
        [-1.4982e-12,  1.1158e-15,  4.3810e-17, -1.0000e+00],
        [-4.8349e-13, -9.2381e-16,  1.2161e-16,  1.9536e-15],
        [-2.6868e-13,  3.4679e-16,  1.0824e-17,  2.0539e-16]]))

In [22]:
B=A
padB = [[0],[0],[0],[0],[0],[0]]
C = np.pad(B, ((0,0),(0,1)), 'constant', constant_values=0)

In [116]:
np.max([np.sum(C[:,i]) for i in range(C.shape[1])])

6

In [129]:
print(A)

[[1 0 0 1 0 0]
 [0 2 0 1 0 0]
 [0 0 3 0 0 0]
 [0 0 0 4 0 0]
 [0 0 0 0 5 0]
 [0 0 0 0 0 6]]


In [128]:
0.32563062*0.32563062+ 0.486049070+.48604907+ 0.06497204*0.06497204 + 0.04622517*0.04622517 + 0.45437627*0.45437627 + 0.66701204*0.66701204

1.7358544292489493

In [55]:

W=np.random.rand(6,2)
print(W)
print(A[:,2:4])
print(A[:,2:4].T @ W)

[[0.80231364 0.51122494]
 [0.55727141 0.54403408]
 [0.98821887 0.34084358]
 [0.2798732  0.66829121]
 [0.93634563 0.19028284]
 [0.96706293 0.30129424]]
[[0 1]
 [0 1]
 [3 0]
 [0 4]
 [0 0]
 [0 0]]
[[2.96465661 1.02253073]
 [2.47907785 3.72842387]]


In [ ]:
A[:,]

In [54]:
B=A
print(B)
B[0:2,3] = np.array([1, 1])
B

[[1 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 3 0 0 0]
 [0 0 0 4 0 0]
 [0 0 0 0 5 0]
 [0 0 0 0 0 6]]


array([[1, 0, 0, 1, 0, 0],
       [0, 2, 0, 1, 0, 0],
       [0, 0, 3, 0, 0, 0],
       [0, 0, 0, 4, 0, 0],
       [0, 0, 0, 0, 5, 0],
       [0, 0, 0, 0, 0, 6]])

In [ ]:
t = x
for c in range(A.shape[1]):
    v = A[:,c].T
    v = v.reshape(v.size,1)
    t = t - v * np.dot(v.T, t)[0][0]/np.dot(v.T,v)
    print('dot product',np.dot(v.T,t)[0][0])

In [ ]:
linalg.norm(x)

In [ ]:
s = 0
for a in x:
    s += a[0]*a[0]
np.sqrt(s)

In [ ]:
W[:,0]

In [ ]:
r